# Video Based classification with cnn-rnn(gru)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

from tensorflow import keras
import imageio
import tensorflow as tf
from tensorflow_docs.vis import embed
import cv2

In [3]:
train_dataset_path = os.listdir("D:\DataSetfor MLandDL\Video classcification\\train")
train_label_types = os.listdir("D:\DataSetfor MLandDL\Video classcification\\train")

In [4]:
print(train_label_types)

['dancing', 'exercise', 'yoga']


# First prepare Training data

In [5]:
# list
rooms=[]
# get all file names
for item in train_dataset_path:
    all_rooms = os.listdir("D:\DataSetfor MLandDL\Video classcification\\train" + '/' + item)
#     add them to list
    for room in all_rooms:
        rooms.append((item, str("D:\DataSetfor MLandDL\Video classcification\\train" + '/' + item) + '/' + room))
                                
# build DataFrame with the help of pandas
train_dataframe = pd.DataFrame(data=rooms,columns=['tags', 'video_name'])
print(train_dataframe.head)
print(train_dataframe.tail)

<bound method NDFrame.head of        tags                                         video_name
0   dancing  D:\DataSetfor MLandDL\Video classcification\tr...
1   dancing  D:\DataSetfor MLandDL\Video classcification\tr...
2   dancing  D:\DataSetfor MLandDL\Video classcification\tr...
3   dancing  D:\DataSetfor MLandDL\Video classcification\tr...
4   dancing  D:\DataSetfor MLandDL\Video classcification\tr...
..      ...                                                ...
93     yoga  D:\DataSetfor MLandDL\Video classcification\tr...
94     yoga  D:\DataSetfor MLandDL\Video classcification\tr...
95     yoga  D:\DataSetfor MLandDL\Video classcification\tr...
96     yoga  D:\DataSetfor MLandDL\Video classcification\tr...
97     yoga  D:\DataSetfor MLandDL\Video classcification\tr...

[98 rows x 2 columns]>
<bound method NDFrame.tail of        tags                                         video_name
0   dancing  D:\DataSetfor MLandDL\Video classcification\tr...
1   dancing  D:\DataSetfor MLandDL

In [6]:
# it means we have total 57 video in train dataset(dance, yoga, exercise)
print(len(train_dataframe))

98


# convert it into csv


In [7]:
df = train_dataframe.loc[:,['video_name','tags']]


In [8]:
df.to_csv('train.csv')

# Preparing Test data

In [9]:
test_dataset_path = os.listdir("D:\DataSetfor MLandDL\Video classcification\\test")
test_label_types = os.listdir("D:\DataSetfor MLandDL\Video classcification\\test")
print(test_label_types)

['dancing', 'exercise', 'yoga']


In [10]:
# empty list
rooms = []

# Get all file name
for item in test_label_types:
    all_rooms = os.listdir("D:\DataSetfor MLandDL\Video classcification\\test" + '/' + item)
    for room in all_rooms:
        rooms.append((item, str("D:\DataSetfor MLandDL\Video classcification\\test" + '/' + item) + '/' + room))

# build dataframe with help of pandas
test_datafram = pd.DataFrame(data=rooms, columns=['tags', 'video_name'])
test_datafram.to_csv('test.csv')

In [14]:
print(test_datafram)

        tags                                         video_name
0    dancing  D:\DataSetfor MLandDL\Video classcification\te...
1    dancing  D:\DataSetfor MLandDL\Video classcification\te...
2    dancing  D:\DataSetfor MLandDL\Video classcification\te...
3    dancing  D:\DataSetfor MLandDL\Video classcification\te...
4   exercise  D:\DataSetfor MLandDL\Video classcification\te...
5   exercise  D:\DataSetfor MLandDL\Video classcification\te...
6   exercise  D:\DataSetfor MLandDL\Video classcification\te...
7   exercise  D:\DataSetfor MLandDL\Video classcification\te...
8   exercise  D:\DataSetfor MLandDL\Video classcification\te...
9   exercise  D:\DataSetfor MLandDL\Video classcification\te...
10      yoga  D:\DataSetfor MLandDL\Video classcification\te...
11      yoga  D:\DataSetfor MLandDL\Video classcification\te...
12      yoga  D:\DataSetfor MLandDL\Video classcification\te...
13      yoga  D:\DataSetfor MLandDL\Video classcification\te...


In [20]:
# !pip install git+https://github.com/tensorflow/docs

^C


# Data preparation for train

In [15]:
train_df = pd.read_csv('D:\DataSetfor MLandDL\Video classcification\\train.csv')
test_df = pd.read_csv('D:\DataSetfor MLandDL\Video classcification\\test.csv')

In [16]:
print(f"Total video for train: {len(train_df)}")
print(f"Total video for train: {len(test_df)}")

Total video for train: 98
Total video for train: 14


# Feed video's to network(means extract frame(sequence of images) from video)

In [25]:
IMAGE_SIZE = 224

# frame crop
def crop_center_square(frame):
    y,x = frame.shape[0:2]
    min_dim = min(y,x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y: start_y + min_dim, start_x : start_x + min_dim]

# video load
def load_video(path ,max_frame=0, resize=(IMAGE_SIZE, IMAGE_SIZE)):
    cap = cv2.VideoCapture(path)
#     store frames which is extracted from the video
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame,resize)
            frame = frame[: ,: , [2,1,0]]
            frames.append(frame)
            
            if len(frames) == max_frame:
                break
            
        
    finally:
        cap.release()
    return np.array(frames)
        
    
    

# Feature Extraction(from the frame that we extracted from video) through CNN

In [26]:
# with the help of this function extract feature from image(frame) that we extracted from video
def build_feature_extraction():
#     used already trained(Inception model) on imagenet dataset
    feature_extractor = keras.applications.InceptionV3(
        weights='imagenet',
        include_top = False,
        pooling = 'avg',
        input_shape = (IMAGE_SIZE,IMAGE_SIZE,3)
    
    )
    preprecess_input = keras.applications.inception_v3.preprocess_input
    input_images = keras.Input((IMAGE_SIZE,IMAGE_SIZE,3))
    preprocessed = preprecess_input(input_images)
    out_put = feature_extractor(preprocessed)
    return keras.Model(input_images, out_put, name="feature_extractor")


feature_extractor = build_feature_extraction()

# Label Encoding(dancing,exercise,yoga)--->gave label to all image

In [18]:
label_preprocess = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(train_dataframe['tags']))
print(label_preprocess)
print(label_preprocess.get_vocabulary())

['dancing', 'exercise', 'yoga']


In [19]:
label = train_dataframe['tags'].values
label = label_preprocess(label[..., None]).numpy()

In [36]:
label


array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2]], dtype=int64)

In [21]:
# Define Hyper-parameter  for model
IMAGE_SIZE =224
BATCH_SIZE = 64
EPOCHS=20
MAX_SEQ_LENGHT = 20 
NUM_FEATURE = 2048


In [38]:
# prepare all videos (train or test)
def prepare_all_video(dataframe, root_dir):
    num_samples = len(dataframe)
    video_path = dataframe['video_name'].values.tolist()
    
#take all class label from train_dataframe and store it label
    labels = dataframe['tags'].values
#convert class label to label encoding
    labels = label_preprocess(labels[...,None]).numpy()

#framemask and framefeature are what we will need feed our sequence model
#frame maske contain a batch of boolean denoting if a timestamp is masked with padding or not
    frame_mask = np.zeros(shape = (num_samples, MAX_SEQ_LENGHT), dtype='bool')   #98,20
    frame_feature = np.zeros(shape = (num_samples,MAX_SEQ_LENGHT,NUM_FEATURE), dtype='float32')  #98,20,2048
    
    
#for each video
    for idx,path in enumerate(video_path):
        
#Gather all its frame and add a batch dimension
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]
        
#initialzed placeholder to store the mask and feature of the current video
        temp_frame_mask = np.zeros(shape=(1,MAX_SEQ_LENGHT,),dtype='bool')
        temp_frame_features = np.zeros(shape =(1, MAX_SEQ_LENGHT, NUM_FEATURE),dtype='float32')
#extract features from the frames of the current videos
        for i, batch in enumerate(frames):
            video_lenght = batch.shape[0]
            lenght = min(MAX_SEQ_LENGHT, video_lenght)
            
            for j in range(lenght):
                temp_frame_features[i,j,:] = feature_extractor.predict(batch[None, j,:])
            temp_frame_mask[i, :lenght] = 1 # 1=not masked , 0=masked
        frame_feature[idx,] = temp_frame_features.squeeze()
        frame_mask[idx,] = temp_frame_mask.squeeze()
        
    return (frame_feature, frame_mask),labels
                
                
                
train_data, train_labels = prepare_all_video(train_dataframe,'train')
test_data, test_labels = prepare_all_video(test_datafram,  'test')
    
    
print(f"Frame Feature in train set :{train_data[0].shape}")
print(f"Frame mask in train set: {train_data[1].shape}")
print(f"Train label in train set: {train_labels.shape}")
print(f"Test label in test set: {test_labels.shape}")

1/1 [==============================] - 0s 323ms/step
Frame Feature in train set :(98, 20, 2048)
Frame mask in train set: (98, 20)
Train label in train set: (98, 1)
Test label in test set: (14, 1)


# Build RNN(GRU)
### Feed the data into GRU to train the model 

In [42]:
print(test_data[0])

[[[1.8546313  1.126411   2.7265856  ... 1.0956511  2.0498095  0.5904637 ]
  [1.2942014  1.3191684  2.6032104  ... 0.51574767 0.68187845 0.36332127]
  [1.9911537  0.8569503  2.8301742  ... 0.13028193 1.045544   1.0456659 ]
  ...
  [0.27543384 1.2533884  1.4120228  ... 1.6304916  1.8101666  1.414971  ]
  [0.25204062 0.9814947  1.5724454  ... 1.3750563  1.3489635  0.86510605]
  [0.5218198  0.70833457 1.4445732  ... 0.9512111  0.9191846  1.7040854 ]]

 [[0.52223897 0.11949713 0.61056614 ... 1.3075376  1.6285411  0.        ]
  [0.50694174 0.13747908 0.7315236  ... 1.2181326  1.4360242  0.        ]
  [0.7679667  0.12446646 0.8056575  ... 1.4754643  1.5453686  0.        ]
  ...
  [0.562966   0.30789298 1.0912538  ... 0.8102498  1.9925206  0.00292508]
  [0.5669049  0.27322164 1.0690475  ... 1.0513803  1.7854825  0.        ]
  [0.33953655 0.25757274 0.86236084 ... 1.1729474  1.5676972  0.01320861]]

 [[0.3929258  1.4366151  0.5545877  ... 0.19790131 0.57277167 0.33373573]
  [0.24085556 1.480507

In [41]:
print(train_data[0])

[[[2.00378299e-01 1.69940174e+00 1.59622487e-02 ... 5.09306312e-01
   1.04805315e-02 0.00000000e+00]
  [2.01448649e-01 1.70672381e+00 1.51906470e-02 ... 5.05703449e-01
   1.06074931e-02 0.00000000e+00]
  [2.31836244e-01 1.70917773e+00 2.37748697e-02 ... 5.42038441e-01
   1.39616886e-02 0.00000000e+00]
  ...
  [2.96424955e-01 1.21741259e+00 5.14605045e-02 ... 5.06625593e-01
   0.00000000e+00 0.00000000e+00]
  [5.26315093e-01 1.14218354e+00 4.52017300e-02 ... 2.78184980e-01
   0.00000000e+00 0.00000000e+00]
  [3.60927224e-01 1.56778502e+00 5.78570589e-02 ... 3.69935572e-01
   0.00000000e+00 7.28861615e-03]]

 [[4.22981858e-01 9.37158614e-02 5.44075847e-01 ... 0.00000000e+00
   0.00000000e+00 8.13956633e-02]
  [4.78551030e-01 5.09395115e-02 6.02171540e-01 ... 5.53833786e-03
   1.07949926e-03 2.28488464e-02]
  [7.03161776e-01 4.49200422e-02 6.90311193e-01 ... 0.00000000e+00
   4.56174370e-03 5.61899804e-02]
  ...
  [5.26067674e-01 2.42515430e-01 7.01067507e-01 ... 3.54766935e-01
   0.00000

In [43]:
# utility for our sequencial model 
def get_sequence_model():
    class_vocab = label_preprocess.get_vocabulary()
    frame_feature_input = keras.Input((MAX_SEQ_LENGHT,NUM_FEATURE))
    mask_input = keras.Input((MAX_SEQ_LENGHT,), dtype='bool')
    x = keras.layers.GRU(16,return_sequences=True)(frame_feature_input, mask = mask_input)
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation='relu')(x)
    out_put= keras.layers.Dense(len(class_vocab), activation='softmax')(x)
    
    gru_model = keras.Model([frame_feature_input, mask_input],out_put)
    gru_model.compile(loss='sparse_categorical_crossentropy', optimizer="adam", metrics=["accuracy"])
    
    return gru_model

# utility for running our gru-model:
epochs = 30

def run_experiment():
    filepath = './temp/video_classifier'
    checkpoint = keras.callbacks.ModelCheckpoint(filepath, verbose=1, save_weights_only=True, save_best_only=True)
    
    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs = epochs,
        callbacks=[checkpoint],
#         validation_data=test_data
    
    )
    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model
    

    
    
_, sequence_model = run_experiment() 


Epoch 1/30
2/3 [===================>..........] - ETA: 0s - loss: 1.1158 - accuracy: 0.2188 
Epoch 1: val_loss improved from inf to 1.27498, saving model to ./temp\video_classifier
3/3 [==============================] - 9s 1s/step - loss: 1.1059 - accuracy: 0.2353 - val_loss: 1.2750 - val_accuracy: 0.0000e+00
Epoch 2/30
2/3 [===================>..........] - ETA: 0s - loss: 0.9861 - accuracy: 0.5156
Epoch 2: val_loss did not improve from 1.27498
3/3 [==============================] - 0s 62ms/step - loss: 0.9919 - accuracy: 0.5147 - val_loss: 1.2824 - val_accuracy: 0.0667
Epoch 3/30
2/3 [===================>..........] - ETA: 0s - loss: 0.9548 - accuracy: 0.5938
Epoch 3: val_loss did not improve from 1.27498
3/3 [==============================] - 0s 61ms/step - loss: 0.9555 - accuracy: 0.6029 - val_loss: 1.3016 - val_accuracy: 0.0333
Epoch 4/30
2/3 [===================>..........] - ETA: 0s - loss: 0.9101 - accuracy: 0.7031
Epoch 4: val_loss improved from 1.27498 to 1.21975, saving mode

In [64]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGHT,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGHT, NUM_FEATURE), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGHT, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(path):
    class_vocab = label_preprocess.get_vocabulary()

    frames = load_video(os.path.join("test", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames

test_video = np.random.choice(test_datafram["video_name"].values.tolist())
print(f"Test video path: {test_video}")

test_frames = sequence_prediction(test_video)

Test video path: D:\DataSetfor MLandDL\Video classcification\test/dancing/pexels-rodnae-productions-7334800.mp4
1/1 [==============================] - 0s 17ms/step
  dancing: 51.01%
  exercise: 29.97%
  yoga: 19.02%


In [61]:
from IPython.display import HTML



HTML("""
    <video alt="test" width="520" height="440" controls>
        <source src="D:\DataSetfor MLandDL\Video classcification\test\yoga\video (1).mp4" type="video/mp4" style="height:300px;width:300px">
    </video>
""")

# HTML("""
    
#     <video alt="test" width="520" height="440" controls
#     <source src="D:\DataSetfor MLandDL\Video classcification\test/dancing/pexels-rodnae-productions-7334800.mp4"
#     video/>
    
# """)